In [1]:
import sys
import rosbag
import time
import rospy
import os
import argparse
import math
import re
import h5py
import pickle as pkl
import numpy as np
import ros_numpy
import matplotlib.pyplot as plt
import tensorflow as tf
from sensor_msgs.msg import Image, PointCloud2, PointField
import open3d as o3d
from tqdm import tqdm, trange

## user define

In [2]:
date = '0805_1447'
folder = '/media/ray/intelSSD/mmdemo_' + date + '/dataset/'
save_folder = '/media/ray/intelSSD/mmdemo_'+date + '/transition_' + date +'/'
files = os.listdir(folder)[::-1]
files.sort()

try:
    os.mkdir(save_folder)
except:
    pass

transition_fname = 'mmdemo_trans_'+date

visualize = False

In [3]:
def reward(laser,action,pos,end):
    done = False
    angular_factor = (1-abs(action[1])) * 5
    r_act = pow(2, angular_factor) * action[0] * 0.2

    r_pos = 10 if np.linalg.norm(pos) > 0.04 else -10

    r_collision = 0
    min_dis = np.min(laser)
    if min_dis < 0.75:
        done = True
        r_collision = -50

    reward = r_act + r_collision + r_pos

    if reward > 0:
        reward = math.log(1+abs(reward))
    elif reward < 0:
        reward = -1 * math.log(1+abs(reward))
    
    if end and not done:
        done = True
        reward += 50

    return reward, done

In [4]:
t_files = tqdm(files)

step_per_file = 512
epi = 0
previous_mm = None
previous_laser = None
previous_odom = None
previous_pos = None
transitions = []
transition_ready = False
sti = 0

for fname in t_files:
    f = h5py.File(folder+fname, 'r+')
    traj_len = f['metadata'].attrs['trajectory_length']

    for i,(mm_scan, action, scan, odom) in enumerate(zip(f['mmwave_scan'][:traj_len], f['action'][:traj_len], f['laser_scan'][:traj_len], f['odom'][:traj_len])):
        r = 0
        if transition_ready:
            sti += 1
            end = (sti==step_per_file)

            pos = odom - previous_odom
            r, done = reward(previous_laser,action,pos,end)
            tran = dict(
                mm_scan=previous_mm,
                laser_scan=previous_laser,
                pos_diff=previous_pos,
                action=action,
                reward=r,
                next_mm_scan=mm_scan,
                next_laser_scan=scan,
                next_pos_diff=pos,
                done=end,                 # asume all transition is not done
            )
            transitions.append(tran)

            if end:
                sti = 0
                with open(save_folder+transition_fname+'_%03d.pkl'%epi,'wb') as f:
                    pkl.dump(transitions, f)
                transitions = []
                epi += 1
            
        t_files.set_description('file: %s, transition: %d reward: %.3f'%(fname, sti, r))
        previous_mm = mm_scan
        previous_laser = scan
        
        if previous_odom is not None:
            previous_pos = odom - previous_odom
            transition_ready = True
            
        previous_odom = odom
        
    f.close()
    del f

if len(transitions) > 0:
    transitions[-1]['done'] = True
    transitions[-1]['reward'] += 50
    with open(save_folder+transition_fname+'_%03d.pkl'%epi,'wb') as pklf:
        pkl.dump(transitions, pklf)
    
print('\nfinish')

file: mmdemo_0805_1425_008.hdf5, transition: 384 reward: -2.398: 100%|██████████| 9/9 [00:06<00:00,  1.33it/s]



finish


## test pkl

In [5]:
test = True
if test:
    with open(save_folder+transition_fname+'_%03d.pkl'%epi,'rb') as f:
        data = pkl.load(f)
        for i,(key,value) in enumerate(data[-2].items()):
            print(key,value)

        print('====================================================================================')
        print('====================================================================================')

        for i,(key,value) in enumerate(data[-1].items()):
            print(key,value)

('mm_scan', array([5.       , 5.       , 5.       , 5.       , 5.       , 5.       ,
       5.       , 5.       , 5.       , 5.       , 5.       , 5.       ,
       5.       , 5.       , 5.       , 5.       , 5.       , 5.       ,
       5.       , 5.       , 5.       , 5.       , 5.       , 5.       ,
       5.       , 5.       , 5.       , 5.       , 5.       , 5.       ,
       5.       , 5.       , 5.       , 5.       , 5.       , 5.       ,
       5.       , 5.       , 5.       , 5.       , 5.       , 5.       ,
       5.       , 5.       , 5.       , 5.       , 5.       , 5.       ,
       5.       , 5.       , 5.       , 5.       , 5.       , 5.       ,
       5.       , 5.       , 5.       , 5.       , 5.       , 5.       ,
       5.       , 5.       , 5.       , 5.       , 5.       , 5.       ,
       5.       , 5.       , 5.       , 5.       , 5.       , 5.       ,
       5.       , 5.       , 5.       , 5.       , 5.       , 5.       ,
       5.       , 5.       , 5.       ,